In [25]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, SMOL_LM_135M, GRPO_MATH_CONFIG

cfg = GRPO_MATH_CONFIG
cfg.notebook_mode = True
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_samples = 1000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()
wrapper.init_data_module()

2025-03-22 14:30:18.936 | INFO     | trl_wrapper.trainer_wrapper:init_model:257 - Using device: mps
2025-03-22 14:30:18.937 | INFO     | trl_wrapper.trainer_wrapper:init_model:271 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa
2025-03-22 14:30:19.390 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:297 - Using chat template override: smollmv2
2025-03-22 14:30:19.391 | INFO     | trl_wrapper.wrapper_config:__init__:126 - Cache dir: ../dataset_caches/g_s_m8_k_data_module


In [28]:
wrapper.init_trainer()

2025-03-22 14:30:21.898 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:334 - Saving output to: ./runs/03-22-14-30-940771-smollm2-135m-instruct-
2025-03-22 14:30:21.900 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:357 - Initializing trainer, run_name: 03-22-14-30-940771-smollm2-135m-instruct-, wandb project: qwen-math-grpo
2025-03-22 14:30:21.900 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:442 - Using vllm: False


In [29]:
wrapper.train()

2025-03-22 14:30:23.875 | INFO     | trl_wrapper.trainer_wrapper:train:610 - Starting training.
2025-03-22 14:30:40.203 | INFO     | model.reasoning:xmlcount_reward_func:83 - XML count rewards: [0.0, 0.0, 0.0, 0.0]
2025-03-22 14:30:40.203 | INFO     | model.reasoning:soft_format_reward_func:61 - Soft format rewards: [0.0, 0.0, 0.0, 0.0]
2025-03-22 14:30:40.204 | INFO     | model.reasoning:strict_format_reward_func:51 - Strict format rewards: [0.0, 0.0, 0.0, 0.0]
2025-03-22 14:30:40.204 | INFO     | model.reasoning:int_reward_func:41 - Integer rewards: [0.0, 0.0, 0.0, 0.0]
2025-03-22 14:30:40.205 | INFO     | model.reasoning:correctness_reward_func:20 - --------------------
Question:
Thomas, Toby, and Rebecca worked a total of 157 hours in one week.  Thomas worked x hours.  Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours less than Toby.  How many hours did Rebecca work?
Answer:
56
Response:
To find out how many hours Rebecca worked, we need to combine

KeyboardInterrupt: 